<a href="https://colab.research.google.com/github/Udbhav-Dalavai/Matrix-completion/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(igraph)
library(mclust)
library(randnet)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Package 'mclust' version 6.1.1
Type 'citation("mclust")' for citing this R package in publications.

Loading required package: Matrix

Loading required package: entropy

Loading required package: AUC

AUC 0.3.2

Type AUCNews() to see the change log and ?AUC to get an overview.

Loading required package: sparseFLMM

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.9-1. For overview type 'help("mgcv-package")'.


Attaching package: ‘mgcv’


The following object is masked from ‘package:mclust’:

    mvn


Loading required package: refund



In [7]:
install.packages('randnet')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘mvtnorm’, ‘locfit’, ‘ash’, ‘FNN’, ‘kernlab’, ‘multicool’, ‘pcaPP’, ‘hdrcde’, ‘ks’, ‘bitops’, ‘rainbow’, ‘RCurl’, ‘fds’, ‘deSolve’, ‘abind’, ‘minqa’, ‘nloptr’, ‘fda’, ‘magic’, ‘gamm4’, ‘lme4’, ‘RLRsim’, ‘grpreg’, ‘pbs’, ‘refund’, ‘RcppEigen’, ‘entropy’, ‘AUC’, ‘sparseFLMM’, ‘poweRlaw’, ‘RSpectra’, ‘irlba’, ‘pracma’, ‘nnls’




In [11]:
# Define the stochastic block model parameters
p1 <- matrix(c(0.5, 0.6,
               0.6, 0.5),
             nrow = 2, byrow = TRUE)

sizes <- c(40, 60)  # Block sizes
d <- 2                  # Embedding dimension
n <- 100
# Generate stochastic block model graph and adjacency matrix
g <- sample_sbm(sum(sizes), pref.matrix = p1, block.sizes = sizes, directed = FALSE)
#A <- as.matrix(as_adjacency_matrix(g, sparse = FALSE))

dt <- BlockModel.Gen(30,n=n,K=3,beta=0.2,rho=0.9,simple=FALSE,power=TRUE)
A = as.matrix(dt$A)

missing_indices <- sample(1:(n^2), size = floor(0.3 * n^2), replace = FALSE)
A[missing_indices] <- NA
A_svd = A
A_svd[is.na(A_svd)] = 0

# Mask matrix (off-diagonal elements are 1, diagonal elements are 0)
M <- matrix(1, nrow = nrow(A), ncol = ncol(A)) - diag(1, nrow(A))

In [65]:
optimize_likelihood <- function(A, d, g = function(x) 1 / (1 + exp(-x)),
                                tol = 1e-5, max_iter = 1000, lr = 0.001, lambda = 0.1) {
  n <- nrow(A)
  #M <- matrix(1, n, n) - diag(1, n)  # Mask matrix
  M <- ifelse(is.na(A), 0, 1)  # Set 0 for NA, 1 for others
  diag(M) <- 0  # Ensure diagonal elements are 0
  X <- matrix(rnorm(n * d, mean = 0, sd = 0.01), n, d)  # Smaller random initialization
  #X = X_init

  clip <- function(x, eps = 1e-6) {
    pmin(pmax(x, eps), 1 - eps)
  }

  log_likelihood <- function(A, X, g) {
    P <- clip(g(X %*% t(X)))
    sum(M * (A * (X %*% t(X)) + log(1 - P)), na.rm = TRUE) - lambda * sum(X^2)
  }

  gradient <- function(A, X, g, lambda = 0.1) {
    P <- g(X %*% t(X))  # Compute probabilities
    grad <- 2 * ((M * A) - (M * P)) %*% X - 2 * lambda * X  # Include regularization
    return(grad)
  }

  clip_gradient <- function(grad, max_norm = 1) {
    norm <- sqrt(sum(grad^2))
    if (norm > max_norm) {
      grad <- grad * (max_norm / norm)
    }
    return(grad)
  }

  for (iter in 1:max_iter) {
    grad <- gradient(A, X, g, lambda)
    grad <- clip_gradient(grad)  # Apply gradient clipping
    X <- X + lr * grad  # Gradient ascent step

    # Check for convergence
    if (sqrt(sum(grad^2)) < tol) break
  }

  list(X = X, log_likelihood = log_likelihood(A, X, g))
}

# Define the sigmoid function as the link function g(x)
sigmoid <- function(x) {
  1 / (1 + exp(-x))
}

In [66]:
observed <- which(!is.na(A), arr.ind = TRUE)

# Optimize the log-likelihood
result <- optimize_likelihood(A_svd, d = 2, g = sigmoid, tol = 1e-5, max_iter = 1000, lr = 0.01)

# Print results
cat("\nOptimized Latent Positions (X):\n")
print(result$X)

cat("\nFinal Log-Likelihood:\n")
print(result$log_likelihood)




Optimized Latent Positions (X):
              [,1]         [,2]
  [1,] -1.08951945 -0.598972860
  [2,]  0.09012193  0.280483547
  [3,]  0.07745254  0.654799882
  [4,] -0.53814709 -0.168622241
  [5,] -0.32949309 -0.393016172
  [6,]  0.36838518  0.476880010
  [7,]  0.11035626  0.818610262
  [8,]  0.20945251  0.501774095
  [9,] -0.82900875 -0.363847594
 [10,]  1.08276874  0.194255469
 [11,]  0.50491819  0.114767922
 [12,]  0.41203424  0.731989710
 [13,]  0.26886056  0.836175133
 [14,]  0.72078659 -0.168942087
 [15,]  0.75409628 -0.170914779
 [16,] -1.11468685 -0.612636613
 [17,]  0.08804122  0.854018442
 [18,] -0.38019871 -0.345372847
 [19,]  0.84152037 -0.281660905
 [20,] -0.40107356 -0.455988496
 [21,] -0.41719390  0.335898708
 [22,]  0.73382803  0.103482358
 [23,]  0.83851333  0.194915751
 [24,] -0.81071512 -0.782192467
 [25,]  0.07392430  0.544764769
 [26,] -0.86984602 -0.711457226
 [27,] -1.36389925 -0.946475764
 [28,] -1.15052559 -0.748019858
 [29,] -0.67935499 -0.417679926
 [30,] 

In [73]:
X_svd = svd(A_svd)
X_svd_u = X_svd$u[, 1:2]

X_init <- X_svd$u[, 1:2]

gmm_new <- Mclust(result$X)
clusters_new <- gmm_new$classification  # Cluster assignments for X_new

# Perform GMM on X_svd_u
gmm_svd <- Mclust(X_svd_u)
clusters_svd <- gmm_svd$classification  # Cluster assignments for X_svd_u

true_labels <- dt$g

# Compute Adjusted Rand Index (ARI)
ari_new <- adjustedRandIndex(true_labels, clusters_new)
ari_svd <- adjustedRandIndex(true_labels, clusters_svd)

# Print ARI results
print(paste("ARI for X_new (Coordinate Descent):", ari_new))
print(paste("ARI for X_svd_u (SVD):", ari_svd))

[1] "ARI for X_new (Coordinate Descent): 1"
[1] "ARI for X_svd_u (SVD): 0.676812738924705"


In [12]:
optimize_likelihood_new <- function(A, d, g = function(x) 1 / (1 + exp(-x)),
                                tol = 1e-5, max_iter = 100, lr = 0.001, lambda = 0.1) {
  n <- nrow(A)
  X <- matrix(rnorm(n * d, mean = 0, sd = 0.01), n, d)  # Random initialization
  observed <- which(!is.na(A), arr.ind = TRUE)

  clip <- function(x, eps = 1e-6) {
    pmin(pmax(x, eps), 1 - eps)
  }

  log_likelihood <- function(A, X, g, observed) {
    ll <- 0
    for (idx in 1:nrow(observed)) {
      i <- observed[idx, 1]
      j <- observed[idx, 2]
      dot_product <- sum(X[i, ] * X[j, ])
      P <- clip(g(dot_product))
      ll <- ll + (A[i, j] * dot_product + (1 - A[i, j]) * log(1 - P)) - lambda * sum(X[i, ]^2)
    }
    ll
  }

  gradient <- function(A, X, g, lambda, idx) {
    i <- observed[idx, 1]
    j <- observed[idx, 2]
    x_i <- X[i, ]
    x_j <- X[j, ]

    dot_product <- sum(x_i * x_j)
    P <- g(dot_product)

    grad_i <- (A[i, j] - P) * x_j - lambda * x_i
    grad_j <- (A[i, j] - P) * x_i - lambda * x_j

    list(grad_i = grad_i, grad_j = grad_j)
  }

  for (iter in 1:max_iter) {
    for (idx in 1:nrow(observed)) {
      grad <- gradient(A, X, g, lambda, idx)
      i <- observed[idx, 1]
      j <- observed[idx, 2]

      X[i, ] <- X[i, ] + lr * grad$grad_i
      X[j, ] <- X[j, ] + lr * grad$grad_j
    }

    # Optional: Compute log-likelihood to monitor progress
    if (iter %% 10 == 0) {
      ll <- log_likelihood(A, X, g, observed)
      cat("Iteration:", iter, "Log-Likelihood:", ll, "\n")
    }
  }

  list(X = X, log_likelihood = log_likelihood(A, X, g, observed))
}

sigmoid <- function(x) {
  1 / (1 + exp(-x))
}

In [13]:
result <- optimize_likelihood_new(A, d = 2, g = sigmoid, tol = 1e-5, max_iter = 1000, lr = 0.01)

cat("\nOptimized Latent Positions (X):\n")
print(result$X)

cat("\nFinal Log-Likelihood:\n")
print(result$log_likelihood)


Iteration: 10 Log-Likelihood: -3433.134 
Iteration: 20 Log-Likelihood: -3432.919 
Iteration: 30 Log-Likelihood: -3431.239 
Iteration: 40 Log-Likelihood: -3417.841 
Iteration: 50 Log-Likelihood: -3327.649 
Iteration: 60 Log-Likelihood: -3060.479 
Iteration: 70 Log-Likelihood: -2867.885 
Iteration: 80 Log-Likelihood: -2820.296 
Iteration: 90 Log-Likelihood: -2812.312 
Iteration: 100 Log-Likelihood: -2811.055 
Iteration: 110 Log-Likelihood: -2810.856 
Iteration: 120 Log-Likelihood: -2810.824 
Iteration: 130 Log-Likelihood: -2810.819 
Iteration: 140 Log-Likelihood: -2810.818 
Iteration: 150 Log-Likelihood: -2810.818 
Iteration: 160 Log-Likelihood: -2810.818 
Iteration: 170 Log-Likelihood: -2810.818 
Iteration: 180 Log-Likelihood: -2810.818 
Iteration: 190 Log-Likelihood: -2810.818 
Iteration: 200 Log-Likelihood: -2810.818 
Iteration: 210 Log-Likelihood: -2810.818 
Iteration: 220 Log-Likelihood: -2810.818 
Iteration: 230 Log-Likelihood: -2810.818 
Iteration: 240 Log-Likelihood: -2810.818 
I

In [19]:
X_svd = svd(A_svd)
X_svd_u = X_svd$u[, 1:2]

#X_init <- X_svd$u[, 1:2]

gmm_new <- Mclust(result$X)
clusters_new <- gmm_new$classification  # Cluster assignments for X_new

# Perform GMM on X_svd_u
gmm_svd <- Mclust(X_svd_u%*%sqrt(X_svd$d[1:2]))
clusters_svd <- gmm_svd$classification  # Cluster assignments for X_svd_u

true_labels <- dt$g

# Compute Adjusted Rand Index (ARI)
ari_new <- adjustedRandIndex(true_labels, clusters_new)
ari_svd <- adjustedRandIndex(true_labels, clusters_svd)

# Print ARI results
print(paste("ARI for X_new (Coordinate Descent):", ari_new))
print(paste("ARI for X_svd_u (SVD):", ari_svd))

[1] "ARI for X_new (Coordinate Descent): 1"
[1] "ARI for X_svd_u (SVD): 0.579713738437959"
